<a href="https://colab.research.google.com/github/ligiamarcia/InteligenciaArtificial/blob/main/Implementacao%20Trabalho%20Final%20de%20Inteligencia%20Artificial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Universidade Federal do amazonas


![Logo da UFAM](https://2020vestibular.com/wp-content/uploads/2017/12/UFAM-2020-e1570295022130.jpg)

# Trabalho Final da disciplina de Inteligência Artificial
## Ministrada pelo Professor Edjar Mota
*Equipe: Lígia Marcia do Nascimento Teixeira, Eriky Rodrigues de Souza,*
 
*Mateus Mota Nóbrega e Gabriel Tavares Ferreira* 


# Dependencias


In [ ]:
#!pip install keras-rectified-adam
!pip install sklearn
!pip install --upgrade gspread    #para uso do google sheets


In [ ]:
import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
import keras 
# from keras.models import Sequential, Model
# from keras.layers import Dense, Input, concatenate
# from keras.losses import binary_crossentropy
# from keras.utils.vis_utils import plot_model
# from keras_radam import RAdam
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# necessário somente se você quiser rodar no google colab, que é o meu caso. 
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Carregando os dados (GET data)

In [ ]:
#Aqui estaremos usando os valores da spreedsheet do google
from google.colab import auth
auth.authenticate_user()
 
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
data = gc.open('db')  #abrindo a planilha 'db' do google drive
data = pd.DataFrame(page.get_all_records())
data  #guarda os valores originais da base

,nome,female(X),male(X),parent(X_Y),parent(Y_X),female(Y),male(Y),hd(X)
0,Tom,0,1,1,0,1,0,1
1,Bob,0,1,1,0,1,0,1
2,Ann,1,0,1,0,1,0,1
3,Pam,1,0,1,0,0,1,-1
4,Pat,1,0,1,0,0,1,-1
5,Liz,1,0,0,1,0,0,-1
6,Eve,1,0,0,1,0,0,-1
7,Jim,0,1,0,1,0,0,-1


In [ ]:
df = data   #df guarda os valores sem a primeira coluna de nomes
df.drop(['nome'], axis=1, inplace=True)
df

,female(X),male(X),parent(X_Y),parent(Y_X),female(Y),male(Y),hd(X)
0,0,1,1,0,1,0,1
1,0,1,1,0,1,0,1
2,1,0,1,0,1,0,1
3,1,0,1,0,0,1,-1
4,1,0,1,0,0,1,-1
5,1,0,0,1,0,0,-1
6,1,0,0,1,0,0,-1
7,0,1,0,1,0,0,-1


In [ ]:
#Valor de saída da rede = Y , ou seja a última coluna é o output
df_Y = data.iloc[:,6]   # todos os valores de y
df_Y

0    1
1    1
2    1
3   -1
4   -1
5   -1
6   -1
7   -1
Name: hd(X), dtype: int64

In [ ]:
df_X = df  # este dataframe está guardando os valores de input da rede
df_X.drop(['hd(X)'], axis=1, inplace=True)
df_X 

,female(X),male(X),parent(X_Y),parent(Y_X),female(Y),male(Y)
0,0,1,1,0,1,0
1,0,1,1,0,1,0
2,1,0,1,0,1,0
3,1,0,1,0,0,1
4,1,0,1,0,0,1
5,1,0,0,1,0,0
6,1,0,0,1,0,0
7,0,1,0,1,0,0


Precisamos neste momento separar a base de dados


*   Treino
*   Teste



In [ ]:
#O modo split em porcentagem divide a base de dados(treino e teste),test_size recebe a porcentagem da 
#base destinada ao teste a rede neural será treinada com o restante
df_X_train, df_X_test, df_Y_train, df_Y_test = train_test_split(df_X, df_Y, 
                                                    test_size=0.30, 
                                                    random_state=123, 
                                                    stratify=df_Y)
print('df_X_train: \n',df_X_train,'\ndf_Y_train: \n',df_Y_train,'\ndf_X_test: \n',df_X_test,'\ndf_Y_test: \n',df_Y_test)

df_X_train: 
    female(X)  male(X)  parent(X_Y)  parent(Y_X)  female(Y)  male(Y)
2          1        0            1            0          1        0
4          1        0            1            0          0        1
6          1        0            0            1          0        0
0          0        1            1            0          1        0
7          0        1            0            1          0        0 
df_Y_train: 
 2    1
4   -1
6   -1
0    1
7   -1
Name: hd(X), dtype: int64 
df_X_test: 
    female(X)  male(X)  parent(X_Y)  parent(Y_X)  female(Y)  male(Y)
1          0        1            1            0          1        0
3          1        0            1            0          0        1
5          1        0            0            1          0        0 
df_Y_test: 
 1    1
3   -1
5   -1
Name: hd(X), dtype: int64


# Configuração da Rede Neural

In [ ]:
epoch = 1000 #Quantidade de iteracoes ou epocas de treinamento
lr = 0.1 #Taxa de aprendizagem
inputlayer_neurons = df_X_train.shape[1] #numero de caracteristicas na camada de entrada
hiddenlayer_neurons = 6 #numero de neuronios na camada de escondida
output_neurons = 1 #numero de neuronios na camada de saida
activation_function = 'tanh' #escolha da funcao de ativacao
randomic_values = 0 # 1 para valores randomicos de W, 0 para valores inicializados


In [ ]:
#Declaração das opções de função de ativação

funcs = {'sigmoid':  (lambda x: 1./(1 + np.exp(-x)),
                      lambda x: x * (1. - x),1),
            'tanh':  (lambda x: np.tanh(x),
                      lambda x: 1. - x*x,0),
        }
(activate, deriv_activate,normalization) = funcs[activation_function]

In [ ]:
#se função de ativação for 'sigmoid', Y será normalizado para [0 a 1]
if (normalization) :
    df_Y_test = np.array([1.0 if i==1.0 else 0.0 for i in df_Y_test]).reshape((df_Y_test.shape[0],1))
    df_Y_train = np.array([1.0 if i==1.0 else 0.0 for i in df_Y_train]).reshape((df_Y_train.shape[0],1))

In [ ]:
if(randomic_values):
    #weight and bias initialization
    #pesos da camada escondida
    wh=np.random.uniform(size=(inputlayer_neurons,hiddenlayer_neurons))
    #bias da camada escondida
    bh=np.random.uniform(size=(1,hiddenlayer_neurons))
    #pesos da camada de saida
    wout=np.random.uniform(size=(hiddenlayer_neurons,output_neurons))
    #bias da camada de saida
    bout=np.random.uniform(size=(1,output_neurons))
else :
    #inicialização de pesos e bias conforme calulado na tabela
    #pesos da camada escondida
    wh = np.full((inputlayer_neurons,hiddenlayer_neurons), 1.0) # peso da camada escondida todos 1
    #bias da camada escondida
    bh= np.array([1.714285714,1.714285714,1.714285714,1.714285714,0.8571428571,0.8571428571]).reshape((1,hiddenlayer_neurons))
    #pesos da camada de saida
    wout= np.full((hiddenlayer_neurons,output_neurons),-1.0)
    wout[0:2]=1 #peso da camada de saida variando entre 1 e -1, dependendo de clausula positiva ou negativa. 1 = positiva, -1 = negativa.
    #bias da camada de saida
    bout = np.array([4.285714286]).reshape((1,output_neurons))

In [ ]:
print("Weight_out\n",wout)

Weight_out
 [[ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]]


# Treino da Rede Neural

In [ ]:
    #Forward Propogation
    hidden_layer_input1=np.dot(df_X_train,wh) 
    hidden_layer_input=hidden_layer_input1 + bh
    hiddenlayer_activations = activate(hidden_layer_input)
    output_layer_input1=np.dot(hiddenlayer_activations,wout)
    output_layer_input= output_layer_input1+ bout
    output = activate(output_layer_input)
    
    #Backpropagation
    E = df_Y_train-output
    slope_output_layer = deriv_activate(output)
    slope_hidden_layer = deriv_activate(hiddenlayer_activations)
    d_output = E*slope_output_layer ##*lr ##
    Error_at_hidden_layer = d_output.dot(wout.T)
    d_hiddenlayer = Error_at_hidden_layer * slope_hidden_layer
    wout = hiddenlayer_activations.T.dot(d_output) *lr +wout
    bout = np.sum(d_output, axis=0,keepdims=True) *lr +bout
    wh = df_X_train.T.dot(d_hiddenlayer) *lr +wh
    bh = np.sum(d_hiddenlayer, axis=0,keepdims=True) *lr +bh

ValueError: ignored

# Teste da Rede Neural

# Avaliando a Rede Neural